In [1]:
# import packages
import numpy as np
from scipy.io import savemat
from brian2 import *
import networkx as nx
rng = np.random.default_rng()

INFO       Cache size for target 'cython': 4490498257 MB.
You can call clear_cache('cython') to delete all files from the cache or manually delete files in the 'C:\Users\mchini\.cython\brian_extensions' directory. [brian2]


In [2]:
# define function to get the degrees of the nodes
def get_degrees(nPre, nPost, connectivity, lognormal_syn_number):
    ############# INPUTS #############
    # nPre = number of presynaptic neurons
    # nPost = number of postsynaptic neurons
    # connectivity = average connecitivty between pre and post-synaptic neurons
    # lognormal_syn_number = if > 0.5, make the number of synapses of individual neurons log-normally distributed
    ##################################
    
    # create a variable to ensure that the maximum number of post-syn partners does not exceed
    # the number of postsynaptic neurons (a neuron can't have more post-synaptic partners than are available)
    # equally ensure that there are only positive values
    max_degrees = nPost + 1 
    min_degrees = -1
    
    while max_degrees > nPost or min_degrees < 1:
        # depending on input, draw a (log)normal distribution 
        if lognormal_syn_number > 0.5:
            degrees = lognormal(0, 0.5, size = nPre) # the mean of this distribution is e**(1/8)            
        else:
            degrees = normal(1, 1 / 4, size = nPre)
        
        # convert the lognormal dist to one with sum of degrees = nPre * nPost * connectivity
        degrees = degrees / degrees.max()    
        degrees = np.round(degrees / degrees.sum() * nPre * nPost * connectivity)
        # extract min and max of degrees
        max_degrees = degrees.max()
        min_degrees = degrees.min()
    
    return degrees

In [3]:
# define function to get the connectivity matrix
def get_bin_mat(nPre, nPost, connectivity, lognormal_syn_number, prop_syn_number):
    
    # create 4 variables to ensure max in_deg < nPre and max out_deg < nPost and both min are > 0
    max_in = nPre + 1
    max_out = nPost + 1
    min_in = -1
    min_out = -1
    
    while max_in >= nPre or max_out >= nPost or min_in < 0 or min_out < 0:
        
        ###############################################
        # start by building the out-degree distribution
        ###############################################

        # make a new out-degree distribution
        out_deg = get_degrees(nPre, nPost, connectivity, lognormal_syn_number)
        # convert number of out-going connections to a probability distribution (sum = 1)
        norm_deg_pre = out_deg / sum(out_deg)

        #################################
        # then the in-degree distribution
        #################################   
    
        # if nPre is not equal to nPost, nPost is from a different population so there are no constraints
        # if prop_syn_number, then we do not want constraints either
        if nPre != nPost or prop_syn_number < 0.5: 
            in_deg = get_degrees(nPost, nPre, connectivity, lognormal_syn_number)
        else: # if nPre == nPost, the two populations are the same (e.g. E to E connections)

                # make number of incoming connections proportional to norm_deg_pre (same population!)   
                targets = rng.choice(nPost, int(nPre * nPost * connectivity), p = norm_deg_pre)
                in_deg = histogram(targets, bins = int(max(targets)) + 1,
                                   range = (0, max(targets) + 1))[0]

        # now make sure that in_deg.sum() == out_deg.sum() or correct for it
        deg_diff = out_deg.sum() - in_deg.sum()
        if nPre < nPost:
            in_deg[randint(0, len(in_deg))] += deg_diff
        else:
            out_deg[randint(0, len(out_deg))] += - deg_diff
        # control max in and out degree after the correction
        max_in = in_deg.max()
        max_out = out_deg.max()
        min_in = in_deg.min()
        min_out = out_deg.min()
    
    # finally, rely on networkx to make a directed connectivity matrix with the desired in-out degrees
    bin_mat = nx.to_numpy_array(nx.directed_havel_hakimi_graph(in_deg, out_deg.astype(int)))
    return bin_mat

In [4]:
# set variables for LIF model and neural network

########### saving and plotting stuff ###########
root_dir = 'your dir here'
v = 'final'

########### network parameters ###########
n_neurons = 400
PYRsProp = 0.8
nPYRS = int(n_neurons * PYRsProp)
nINs = int(n_neurons - nPYRS)
defaultclock.dt = 0.1 * ms
voltage_clock = Clock(dt=5*ms)               # use different clock to change sampling rate
simulation_time = 10*second
PYRs2keep = 320
INs2keep = 80
n_reps = 8000                                # number of times to repeat simulation
   
# Neuron model
CmE = 0.5 * nF                               # Membrane capacitance of excitatory neurons
CmI = 0.2 * nF                               # Membrane capacitance of inhibitory neurons
gLeakE = 25.0 * nS                           # Leak conductance of excitatory neurons
gLeakI = 20.0 * nS                           # Leak conductance of inhibitory neurons
Vl = -70.0 * mV                              # Leak membrane potential
Vthr = -52.0 * mV                            # Spiking threshold
Vrest = -59.0 * mV                           # Reset potential
refractoryE = 2 * ms                         # refractory period
refractoryI = 1 * ms                         # refractory period
TauE = 20 * ms
TauI = 10 * ms

# Synapse model
VrevE = 0 * mV                               # Reversal potential of excitatory synapses
VrevI = -80 * mV                             # Reversal potential of inhibitory synapses
tau_AMPA_E = 2.0 * ms                        # Decay constant of AMPA-type conductances
tau_AMPA_I = 1.0 * ms                        # Decay constant of AMPA-type conductances
tau_GABA = 5.0 * ms                          # Decay constant of GABA-type conductances

In [5]:
# Neuron equations
eqsPYR = '''
dV/dt = (-gea*(V-VrevE) - gi*(V-VrevI) - (V-Vl)) / (tau) + sigma/tau**.5*xi : volt
dgea/dt = -gea/(tau_AMPA_E) : 1
dgi/dt = -gi/(tau_GABA) : 1
tau : second
Cm : farad
sigma : volt
'''
eqsIN = '''
dV/dt = (-gea*(V-VrevE) - gi*(V-VrevI) - (V-Vl)) / (tau) + sigma/tau**.5*xi: volt
dgea/dt = -gea/(tau_AMPA_I) : 1
dgi/dt = -gi/(tau_GABA) : 1
tau : second
Cm : farad
sigma : volt
'''

In [6]:
#%% now actually run the model

for iRep in range(n_reps):
    
    # set random parameters
    PYRs_sigma = normal(15, 1)
    IN_sigma = normal(12.5, 1)
    AMPA_mod = normal(0.7, 0.7/4)
    GABA_mod = normal(2, 0.5)
    connectivity = normal(0.25, 0.25/4)
    lognormal_syn_weight = rand(1)
    lognormal_syn_number = rand(1)
    prop_syn_number = rand(1)
    
    ########### define neuron groups ###########
    PYRs = NeuronGroup(nPYRS, method='euler',
                       model=eqsPYR,
                       threshold = "V>Vthr",  reset = "V=Vrest",
                       refractory=refractoryE)
    PYRs.Cm = CmE
    PYRs.tau = CmE / gLeakE
    PYRs.sigma = PYRs_sigma * mV
                        
    IN = NeuronGroup(nINs, method='euler',
                     model=eqsIN,
                     threshold = "V>Vthr",  reset = "V=Vrest",
                     refractory=refractoryI)
    IN.Cm = CmI
    IN.tau = CmI / gLeakI      
    IN.sigma = IN_sigma * mV
                    
    # define AMPA and GABA synapses parameters
    Cee = Synapses(PYRs, PYRs, 'w: 1', on_pre='gea+=w')
    Cei = Synapses(PYRs, IN, 'w: 1', on_pre='gea+=w')
    Cie = Synapses(IN, PYRs, 'w: 1', on_pre='gi+=w')
    Cii = Synapses(IN, IN, 'w: 1', on_pre='gi+=w')
    
    # compute all connectivity matrices
    EE_mat = get_bin_mat(nPYRS, nPYRS, connectivity, lognormal_syn_number, prop_syn_number)
    EI_mat = get_bin_mat(nPYRS, nINs, connectivity, lognormal_syn_number, prop_syn_number)
    IE_mat = get_bin_mat(nINs, nPYRS, connectivity, lognormal_syn_number, prop_syn_number)
    II_mat =  get_bin_mat(nINs, nINs, connectivity, lognormal_syn_number, prop_syn_number)
    
    # now connect all synapses
    # PYR to PYR    
    sources, targets = EE_mat.nonzero()
    Cee.connect(i=sources, j=targets)
    Cee.delay = 0 * ms    
    # PYR to IN    
    sources, targets = EI_mat.nonzero()
    Cei.connect(i=sources, j=targets)
    Cei.delay = 0 * ms        
    # IN to PYR    
    sources, targets = IE_mat.nonzero()
    Cie.connect(i=sources, j=targets)
    Cie.delay = 0 * ms                
    # IN to IN    
    sources, targets = II_mat.nonzero()
    Cii.connect(i=sources, j=targets)
    Cii.delay = 0 * ms    
    
    # please note that the two distributions here below have the same mean
    
    # compute synaptic weight distributions
    if lognormal_syn_weight > 0.5:
        gEE = lognormal(0, 1, Cee.w.shape[0]) / 25 * AMPA_mod
        gEI = lognormal(0, 1, Cei.w.shape[0]) / 25 * AMPA_mod
        gIE = lognormal(0, 1, Cie.w.shape[0]) / 6 * GABA_mod
        gII = lognormal(0, 1, Cii.w.shape[0]) / 30 * GABA_mod
    else:
        gEE = normal(sqrt(e), 0.5, Cee.w.shape[0]) / 25 * AMPA_mod
        gEI = normal(sqrt(e), 0.5, Cei.w.shape[0]) / 25 * AMPA_mod
        gIE = normal(sqrt(e), 0.5, Cie.w.shape[0]) / 6 * GABA_mod
        gII = normal(sqrt(e), 0.5, Cii.w.shape[0]) / 30 * GABA_mod

    # set the weight for all the synapses
    Cee.w = gEE
    Cei.w = gEI
    Cie.w = gIE
    Cii.w = gII    

    # initialize voltage
    PYRs.V = Vrest + (rand(nPYRS) * 5 - 5) * mV
    IN.V = Vrest + (rand(nINs) * 5 - 5) * mV

    # record spikes of excitatory neurons
    Sp_E = SpikeMonitor(PYRs, record=True)
    # record spikes of inhibitory neurons
    Sp_I = SpikeMonitor(IN, record=True)

    #------------------------------------------------------------------------------
    # Run the simulation
    #------------------------------------------------------------------------------
    run(simulation_time)

    dict2save = {}
    dict2save['spikes_PYR'] = Sp_E.i, Sp_E.t
    dict2save['spikes_IN'] = Sp_I.i, Sp_I.t
    dict2save['PYRs_sigma'] = PYRs_sigma
    dict2save['IN_sigma'] = IN_sigma
    dict2save['AMPA_mod'] = AMPA_mod
    dict2save['GABA_mod'] = GABA_mod
    dict2save['connectivity'] = connectivity
    dict2save['lognormal_syn_weight'] = lognormal_syn_weight
    dict2save['lognormal_syn_number'] = lognormal_syn_number
    dict2save['prop_syn_number'] = prop_syn_number

    savemat(root_dir + str(v) + '_iRep_' + str(iRep) + '.mat', dict2save)

    print('done with simulation ' + str(iRep))

    del Sp_E, Sp_I

done with simulation 7621
done with simulation 7622
done with simulation 7623
done with simulation 7624
done with simulation 7625
done with simulation 7626
done with simulation 7627
done with simulation 7628
done with simulation 7629
done with simulation 7630
done with simulation 7631
done with simulation 7632
done with simulation 7633
done with simulation 7634
done with simulation 7635
done with simulation 7636
done with simulation 7637
done with simulation 7638
done with simulation 7639
done with simulation 7640
done with simulation 7641
done with simulation 7642
done with simulation 7643
done with simulation 7644
done with simulation 7645
done with simulation 7646
done with simulation 7647
done with simulation 7648
done with simulation 7649
done with simulation 7650
done with simulation 7651
done with simulation 7652
done with simulation 7653
done with simulation 7654
done with simulation 7655
done with simulation 7656
done with simulation 7657
done with simulation 7658
done with si

done with simulation 7937
done with simulation 7938
done with simulation 7939
done with simulation 7940
done with simulation 7941
done with simulation 7942
done with simulation 7943
done with simulation 7944
done with simulation 7945
done with simulation 7946
done with simulation 7947
done with simulation 7948
done with simulation 7949
done with simulation 7950
done with simulation 7951
done with simulation 7952
done with simulation 7953
done with simulation 7954
done with simulation 7955
done with simulation 7956
done with simulation 7957
done with simulation 7958
done with simulation 7959
done with simulation 7960
done with simulation 7961
done with simulation 7962
done with simulation 7963
done with simulation 7964
done with simulation 7965
done with simulation 7966
done with simulation 7967
done with simulation 7968
done with simulation 7969
done with simulation 7970
done with simulation 7971
done with simulation 7972
done with simulation 7973
done with simulation 7974
done with si